In [40]:
from os.path import abspath
from pyspark.sql import SparkSession

# set default location for [spark-warehouse]
warehouse_location = abspath('spark-warehouse')

In [41]:
spark = (
    SparkSession
    .builder
    .appName("Delta Lake")
    .config("spark.jars.packages", "org.apache.spark:spark-avro_2.12:3.1.1,io.delta:delta-core_2.12:1.0.0,org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.1")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .config("spark.sql.debug.maxToStringFields","1000")
    .config("spark.kryoserializer.buffer.max","128m")
    .config("spark.driver.memory", "16G")
    .enableHiveSupport()
    .getOrCreate()
)
spark.sparkContext.setLogLevel("WARN")

In [42]:
conf = spark.sparkContext._jsc.hadoopConfiguration()
conf.set("fs.gs.project.id", "hering-datalake-prod")
conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
conf.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")

---

In [43]:
from pyspark.sql.avro.functions import from_avro, to_avro
import pyspark.sql.functions as f
from urllib.request import urlopen
import json
from pyspark.sql.types import StringType
from dateutil import parser
from pyspark.sql import DataFrame as SparkDataFrame,DataFrame
from datetime import datetime
from typing import List
from dateutil import parser
import json
import pyspark.sql.functions as f
from pyspark.sql.types import StructField, StructType, StringType, DoubleType, BinaryType, IntegerType, TimestampType
from pyspark.sql import Column
from pyspark.sql import DataFrame as SparkDataFrame,DataFrame
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType
from delta.tables import DeltaTable

In [44]:

def generete_symlink_manifest(spark:SparkSession, bucket:str) -> None:
        """
        Gera o arquivo symlink manifest para a tabela Delta do bucket de referencia
        """
        delta_table = DeltaTable.forPath(spark, bucket)
        delta_table.generate("symlink_format_manifest")


def path_exists(path, spark:SparkSession):
        """
        Dado um caminho verifica se existe ou não
        """
        sc = spark.sparkContext
        fs = sc._jvm.org.apache.hadoop.fs.FileSystem.get(
            sc._jvm.java.net.URI.create("gs://" + path.split("/")[2]),
            sc._jsc.hadoopConfiguration(),
        )
        return fs.exists(sc._jvm.org.apache.hadoop.fs.Path(path))


def last_date(delta_path:str, spark:SparkSession):
        """
        Retorna a data (timestamp_kafka) do último registro da tabela Delta
        """
        if not path_exists(delta_path + 'job_control/last_date', spark):
            try:
                df_temp = (
                    spark
                    .read
                    .format("delta")
                    .load(delta_path)
                )

                list_result = (
                    df_temp
                    .agg({"timestamp_kafka": "max"}).collect()
                )

                last_date = str(list_result[0].asDict()['max(timestamp_kafka)'])
                snapshot_flag = False
            except:
                snapshot_flag = True
                last_date = ('01-01-2021 12:00AM')

            df = spark.read.json(spark.sparkContext.parallelize([{'last_date': last_date}]))
            df.write.mode('overwrite').csv(delta_path + 'job_control/last_date', header = 'true')
        else:
            snapshot_flag = False
            df = spark.read.format("csv").load(delta_path + 'job_control/last_date', header = 'true')
            last_date = df.collect()[0].asDict()['last_date']

        return parser.parse(last_date),snapshot_flag


def update_last_date(delta_path:str, df:DataFrame, spark:SparkSession):
        """
        Atualiza a data (timestamp_kafka) do último registro da tabela Delta
        """
        list_result = (
            df
            .agg({"timestamp_kafka": "max"}).collect()
        )

        last_date = str(list_result[0].asDict()['max(timestamp_kafka)'])
        df = spark.read.json(spark.sparkContext.parallelize([{'last_date': last_date}]))
        df.write.mode('overwrite').csv(delta_path + 'job_control/last_date', header = 'true')

        return last_date

def get_operations_counts(df:DataFrame, spark:SparkSession):
        """
        Retorna o número de registros de cada operação kafka (__op)
        """
        op_count_list =(
            df
            .groupBy('__op')
            .count()
            .collect()
        )
        
        new_data_count = 0
        updating_data_count = 0
        deleting_data_count = 0

        if len(op_count_list) > 0:
            for i in op_count_list:
                dic_temp = i.asDict()
                if (dic_temp['__op'] == 'r'):
                    new_data_count = new_data_count + dic_temp['count']
                elif (dic_temp['__op'] != 'r'):
                    updating_data_count = updating_data_count + dic_temp['count']
                elif (dic_temp['__op'] == 'd'):
                    deleting_data_count = deleting_data_count + dic_temp['count']

        new_total_data_count = new_data_count + updating_data_count + deleting_data_count
        
        return new_total_data_count, new_data_count, updating_data_count, deleting_data_count


def get_data(spark:SparkSession, bucket:str, date_colunm_name:str, date_begin:datetime, source_format:str='delta', is_landing:bool=False):
        """
        Carrega Dataframe filtrando os dados do bucket de referência conforme 
        data de início enviada no parametro

        :param str date_begin: Data de início para filtro dos dados
        """
        if is_landing:
            schema = StructType() \
                    .add("key",BinaryType(),True) \
                    .add("value",BinaryType(),True) \
                    .add("topic",StringType(),True) \
                    .add("partition",IntegerType(),True) \
                    .add("offset",StringType(),True) \
                    .add("timestamp",TimestampType(),True)
            df = (
                spark
                .read
                .format(source_format)
                .schema(schema)
                .load(bucket)
                .where(
                    (f.col("year")>date_begin.year)|
                    (
                        (f.col("year")==date_begin.year)&
                        (
                        (f.col("month")>date_begin.month)|
                        (
                            (f.col("month")==date_begin.month)&
                            (
                            (f.col("day")>date_begin.day)|
                            (
                                (f.col("day")==date_begin.day)&
                                (f.col("hour")>=date_begin.hour)&
                                (f.col(date_colunm_name)>date_begin)
                            )
                            )
                        )
                        )          
                    ) 
                )
            )
        else:
            df = (
                spark
                .read
                .format(source_format)
                .load(bucket)
                .where(
                    (f.col("year")>date_begin.year)|
                    (
                        (f.col("year")==date_begin.year)&
                        (
                        (f.col("month")>date_begin.month)|
                        (
                            (f.col("month")==date_begin.month)&
                            (
                            (f.col("day")>date_begin.day)|
                            (
                                (f.col("day")==date_begin.day)&
                                (f.col("hour")>=date_begin.hour)&
                                (f.col(date_colunm_name)>date_begin)
                            )
                            )
                        )
                        )          
                    ) 
                )
            )
        return df


def write_new_data(spark:SparkSession, df:DataFrame, bucket:str, destination_format:str='delta'):
        """
        Escreve no bucket de destino os dados novos, cuja a operação kafka é 'r'
        """
        df_r = df.where((f.col('__op') == 'r'))
        (
            df_r
            .write
            .format(destination_format)
            .mode("append")
            .option("checkpointLocation", bucket + "checkpoint")
            .save(bucket)
        )
        
        return True


def update_new_data(spark:SparkSession, df:DataFrame, bucket:str, pk:list=['pk_single']):
        """
        Escreve no bucket de destino os dados novos, cuja a operação kafka é 'c' e
        atualiza os dados existentes, cuja a operação kafka é 'u' ou 'd'
        """
        table = DeltaTable.forPath(spark, bucket)
        df_table_u = df.where((f.col('__op') != 'r'))
        df_table_u_unique = df_table_u.orderBy('timestamp_kafka', ascending = False).dropDuplicates(pk)
        merge_string = 'destination.' + pk[0] + ' = updates.' + pk[0]
        (
            table.alias('destination')
            .merge(df_table_u_unique.alias('updates'), merge_string)
            .whenMatchedUpdateAll()
            .whenNotMatchedInsertAll()
            .execute()
        )
        
        return True


def delete_new_data(spark:SparkSession, bucket:str):
        """
        Deleta da tabela delta, no bucket de destino, os dados cuja a operação kafka é 'd'
        """
        table = DeltaTable.forPath(spark, bucket)
        table.delete(f.col('__op') == 'd')
        
        return True


def flatten_df(nested_df):
        """
        Retorna um dataframe com as colunas de um dataframe aninhado
        """
        flat_cols = [c[0] for c in nested_df.dtypes if c[1][:6] != 'struct']
        nested_cols = [c[0] for c in nested_df.dtypes if c[1][:6] == 'struct']

        flat_df = nested_df.select(flat_cols +
                                [f.col(nc+'.'+c)
                                    for nc in nested_cols
                                    for c in nested_df.select(nc+'.*').columns])
        return flat_df


def write_bronze_table(df:DataFrame, bucket:str):
        """
        Escreve no bucket de destino a tabela delta bronze
        """
        (
            df
            .write
            .partitionBy("year", "month", "day", "hour")
            .format("delta")
            .mode("append")
            .option("checkpointLocation", bucket + "checkpoint")
            .save(bucket)
        )

        return True

def get_schema(df:DataFrame):
    json_str = df.limit(1).toPandas()['fixedValue'][0]
    json_fields = json.loads(json_str)['schema']['fields']
    struct_fields = []
    for field in json_fields:
        struct_fields.append(StructField(field['field'], StringType()))
        
    return StructType(struct_fields)

def decode_data_frame(df:DataFrame, schema:StructType):
    df_decoded = ( 
        df
        .withColumn('fixedValue', f.col("value").cast("string"))
        .select(f.get_json_object('fixedValue',"$.payload").alias('payload'), 'timestamp_kafka')
        .withColumnRenamed('timestamp', 'timestamp_kafka')
        .select(f.from_json(f.col('payload'), schema).alias("DF"), 'timestamp_kafka')
        .select("DF.*" , 'timestamp_kafka')
    )
    
    return df_decoded

In [45]:
#PATH LANDING ZONE
LANDING_PATH_VENDA = 'gs://hering-datalake-prod-southamerica-east1-bucket-landing-zone/bnsvher602/db_mobile/dbo/venda/'
LANDING_PATH_VENDA_ITEM = 'gs://hering-datalake-prod-southamerica-east1-bucket-landing-zone/bnsvher602/db_mobile/dbo/venda_item/'
LANDING_PATH_FILIAL = 'gs://hering-datalake-prod-southamerica-east1-bucket-landing-zone/bnsvher602/db_mobile/dbo/filial/'
LANDING_PATH_REDE_LOJA = 'gs://hering-datalake-prod-southamerica-east1-bucket-landing-zone/bnsvher602/db_mobile/dbo/rede_loja/'
LANDING_PATH_EMPRESA = 'gs://hering-datalake-prod-southamerica-east1-bucket-landing-zone/bnsvher602/db_mobile/dbo/empresa/'
LANDING_PATH_PRODUTO_BARRA = 'gs://hering-datalake-prod-southamerica-east1-bucket-landing-zone/bnsvher602/db_mobile/dbo/mb_produtos_barra/'
LANDING_PATH_VENDEDOR_LOJA = 'gs://hering-datalake-prod-southamerica-east1-bucket-landing-zone/bnsvher602/db_mobile/dbo/vendedor_loja/'
LANDING_PATH_FILIAL_PONTO = 'gs://hering-datalake-prod-southamerica-east1-bucket-landing-zone/bnsvher602/db_mobile/dbo/filial_ponto/'
LANDING_PATH_SITE = 'gs://hering-datalake-prod-southamerica-east1-bucket-landing-zone/bnsvher602/db_mobile/dbo/site/'
LANDING_PATH_GERENTE_REGIONAL = 'gs://hering-datalake-prod-southamerica-east1-bucket-landing-zone/bnsvher602/db_mobile/dbo/gerente_regional/'
LANDING_PATH_CONSULTOR = 'gs://hering-datalake-prod-southamerica-east1-bucket-landing-zone/bnsvher602/db_mobile/dbo/consultor/'

LANDING_PATH_PRODUTOS =  'gs://hering-datalake-prod-southamerica-east1-bucket-landing-zone/bnsvher602/linx_db/dbo/produtos/'
LANDING_PATH_PRODUTOS_BARRA_LINX = 'gs://hering-datalake-prod-southamerica-east1-bucket-landing-zone/bnsvher602/linx_db/dbo/produtos_barra/'
LANDING_PATH_HR_WEB_CATALOGO_SKU =  'gs://hering-datalake-prod-southamerica-east1-bucket-landing-zone/bnsvher602/linx_db/dbo/hr_web_catalogo_sku/'
LANDING_PATH_HR_WEB_CATALOGO_FOTO =  'gs://hering-datalake-prod-southamerica-east1-bucket-landing-zone/bnsvher602/linx_db/dbo/hr_web_catalogo_foto/'
LANDING_PATH_PRODUTO_CORES =  'gs://hering-datalake-prod-southamerica-east1-bucket-landing-zone/bnsvher602/linx_db/dbo/produto_cores/'

LANDING_PATH_BASE_CLIENTES =  'gs://hering-datalake-prod-southamerica-east1-bucket-landing-zone/bnsvher361/db_analise/crm/tb_base_clientes/'

#PATH BRONZE ZONE
BRONZE_PATH_VENDA = 'gs://hering-datalake-prod-southamerica-east1-bucket-bronze-zone/oto/bnsvher602/db_mobile/dbo/venda/'
BRONZE_PATH_VENDA_ITEM = 'gs://hering-datalake-prod-southamerica-east1-bucket-bronze-zone/oto/bnsvher602/db_mobile/dbo/venda_item/'
BRONZE_PATH_FILIAL = 'gs://hering-datalake-prod-southamerica-east1-bucket-bronze-zone/oto/bnsvher602/db_mobile/dbo/filial/'
BRONZE_PATH_REDE_LOJA = 'gs://hering-datalake-prod-southamerica-east1-bucket-bronze-zone/oto/bnsvher602/db_mobile/dbo/rede_loja/'
BRONZE_PATH_EMPRESA = 'gs://hering-datalake-prod-southamerica-east1-bucket-bronze-zone/oto/bnsvher602/db_mobile/dbo/empresa/'
BRONZE_PATH_PRODUTO_BARRA = 'gs://hering-datalake-prod-southamerica-east1-bucket-bronze-zone/oto/bnsvher602/db_mobile/dbo/mb_produtos_barra/'
BRONZE_PATH_VENDEDOR_LOJA = 'gs://hering-datalake-prod-southamerica-east1-bucket-bronze-zone/oto/bnsvher602/db_mobile/dbo/vendedor_loja/'
BRONZE_PATH_FILIAL_PONTO = 'gs://hering-datalake-prod-southamerica-east1-bucket-bronze-zone/oto/bnsvher602/db_mobile/dbo/filial_ponto/'
BRONZE_PATH_SITE = 'gs://hering-datalake-prod-southamerica-east1-bucket-bronze-zone/oto/bnsvher602/db_mobile/dbo/site/'
BRONZE_PATH_GERENTE_REGIONAL = 'gs://hering-datalake-prod-southamerica-east1-bucket-bronze-zone/oto/bnsvher602/db_mobile/dbo/gerente_regional/'
BRONZE_PATH_CONSULTOR = 'gs://hering-datalake-prod-southamerica-east1-bucket-bronze-zone/oto/bnsvher602/db_mobile/dbo/consultor/'

BRONZE_PATH_PRODUTOS =  'gs://hering-datalake-prod-southamerica-east1-bucket-bronze-zone/oto/bnsvher602/linx_db/dbo/produtos/'
BRONZE_PATH_PRODUTOS_BARRA_LINX = 'gs://hering-datalake-prod-southamerica-east1-bucket-bronze-zone/oto/bnsvher602/linx_db/dbo/produtos_barra/'
BRONZE_PATH_HR_WEB_CATALOGO_SKU =  'gs://hering-datalake-prod-southamerica-east1-bucket-bronze-zone/oto/bnsvher602/linx_db/dbo/hr_web_catalogo_sku/'
BRONZE_PATH_HR_WEB_CATALOGO_FOTO =  'gs://hering-datalake-prod-southamerica-east1-bucket-bronze-zone/oto/bnsvher602/linx_db/dbo/hr_web_catalogo_foto/'
BRONZE_PATH_PRODUTO_CORES =  'gs://hering-datalake-prod-southamerica-east1-bucket-bronze-zone/oto/bnsvher602/linx_db/dbo/produto_cores/'

BRONZE_PATH_BASE_CLIENTES =  'gs://hering-datalake-prod-southamerica-east1-bucket-bronze-zone/oto/bnsvher361/db_analise/crm/tb_base_clientes/'



#PATH SILVER ZONE
SILVER_PATH_VENDA = 'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/oto/bnsvher602/db_mobile/dbo/venda/'
SILVER_PATH_VENDA_ITEM = 'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/oto/bnsvher602/db_mobile/dbo/venda_item/'
SILVER_PATH_FILIAL = 'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/oto/bnsvher602/db_mobile/dbo/filial/'
SILVER_PATH_REDE_LOJA = 'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/oto/bnsvher602/db_mobile/dbo/rede_loja/'
SILVER_PATH_EMPRESA = 'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/oto/bnsvher602/db_mobile/dbo/empresa/'
SILVER_PATH_PRODUTO_BARRA = 'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/oto/bnsvher602/db_mobile/dbo/mb_produtos_barra/'
SILVER_PATH_VENDEDOR_LOJA = 'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/oto/bnsvher602/db_mobile/dbo/vendedor_loja/'
SILVER_PATH_FILIAL_PONTO = 'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/oto/bnsvher602/db_mobile/dbo/filial_ponto/'
SILVER_PATH_SITE = 'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/oto/bnsvher602/db_mobile/dbo/site/'
SILVER_PATH_GERENTE_REGIONAL = 'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/oto/bnsvher602/db_mobile/dbo/gerente_regional/'
SILVER_PATH_CONSULTOR = 'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/oto/bnsvher602/db_mobile/dbo/consultor/'

SILVER_PATH_PRODUTOS =  'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/oto/bnsvher602/linx_db/dbo/produtos/'
SILVER_PATH_PRODUTOS_BARRA_LINX = 'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/oto/bnsvher602/linx_db/dbo/produtos_barra/'
SILVER_PATH_HR_WEB_CATALOGO_SKU =  'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/oto/bnsvher602/linx_db/dbo/hr_web_catalogo_sku/'
SILVER_PATH_HR_WEB_CATALOGO_FOTO =  'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/oto/bnsvher602/linx_db/dbo/hr_web_catalogo_foto/'
SILVER_PATH_PRODUTO_CORES =  'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/oto/bnsvher602/linx_db/dbo/produto_cores/'

SILVER_PATH_BASE_CLIENTES =  'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/oto/bnsvher361/db_analise/crm/tb_base_clientes/'



#PATH GOLD ZONE
GOLD_PATH_CUSTOMERS = 'gs://hering-datalake-prod-southamerica-east1-bucket-gold-zone/oto/homol/customers'
GOLD_PATH_ORDERS = 'gs://hering-datalake-prod-southamerica-east1-bucket-gold-zone/oto/homol/orders/'
GOLD_PATH_ORDERS_ITEMS = 'gs://hering-datalake-prod-southamerica-east1-bucket-gold-zone/oto/homol/orders_items/'
GOLD_PATH_PRODUCTS = 'gs://hering-datalake-prod-southamerica-east1-bucket-gold-zone/oto/homol/products/'
GOLD_PATH_SELLERS = 'gs://hering-datalake-prod-southamerica-east1-bucket-gold-zone/oto/homol/sellers/'
GOLD_PATH_STORES = 'gs://hering-datalake-prod-southamerica-east1-bucket-gold-zone/oto/homol/stores/'

GOLD_PATH_OTO_DZARM = 'gs://hering-datalake-prod-southamerica-east1-bucket-gold-zone/oto/homol/dzarm/'
GOLD_PATH_OTO_HERING_HMG = 'gs://hering-datalake-prod-southamerica-east1-bucket-gold-zone/oto/homol/hering_hmg/'
GOLD_PATH_OTO_HERING_KIDS = 'gs://hering-datalake-prod-southamerica-east1-bucket-gold-zone/oto/homol/hering_kids/'
GOLD_PATH_OTO_HERING_OUTLETS = 'gs://hering-datalake-prod-southamerica-east1-bucket-gold-zone/oto/homol/hering_outlets/'

GOLD_PATH_STORES = 'gs://hering-datalake-prod-southamerica-east1-bucket-gold-zone/oto/homol/stores'

In [26]:
venda_bronze  = DeltaTable.forPath(spark, BRONZE_PATH_VENDA)
venda_item_bronze  = DeltaTable.forPath(spark, BRONZE_PATH_VENDA_ITEM)
venda_silver  = DeltaTable.forPath(spark, SILVER_PATH_VENDA)
venda_item_silver  = DeltaTable.forPath(spark, SILVER_PATH_VENDA_ITEM)

df_table = get_data(spark, LANDING_PATH_VENDA, 'timestamp', last_date, 'json', True)

In [27]:
venda_bronze.toDF().createOrReplaceTempView("view_venda_bronze")
venda_item_bronze.toDF().createOrReplaceTempView("view_venda_item_bronze")
venda_silver.toDF().createOrReplaceTempView("view_venda_silver")
venda_item_silver.toDF().createOrReplaceTempView("view_venda_item_silver")

In [28]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [38]:
#--SELECT YEAR(DATA) AS ANO, COUNT(*) FROM  view_venda GROUP BY  YEAR(DATA)
result_sql_bronze = spark.sql(
'''

SELECT 
YEAR(from_unixtime(unix_timestamp( DATA_DIGITACAO),'yyyy-MM-dd')) as ANO
,count(*) as QTD
FROM  view_venda_silver 
GROUP BY YEAR(from_unixtime(unix_timestamp( DATA_DIGITACAO),'yyyy-MM-dd'))
ORDER BY YEAR(from_unixtime(unix_timestamp( DATA_DIGITACAO),'yyyy-MM-dd')) DESC

''')

In [39]:
result_sql_bronze.toPandas()

ANO       QTD
0   2022   2369690
1   2021   6618460
2   2020   5925028
3   2019   8420854
4   2018   9402526
5   2017  10547445
6   2016  12857652
7   2015        72
8   2008         1
9   2007       137
10  2000         2